#### 📄 Synthetic Data Generation from Turkish Financial PDFs using Gemma3n 🔥🦙
In this project, I focused on generating **synthetic instruction-style data from Turkish financial PDF documents using Gemma3n (via LangChain + Ollama).** My goal was to create a scalable and automated pipeline to enrich the currently limited Turkish-language datasets in the financial domain.

The system takes a PDF as input, extracts contextually relevant segments, and generates meaningful instruction-response pairs using an LLM. This synthetic data can be used to fine-tune financial chatbots or support RAG (Retrieval-Augmented Generation) systems for improved question answering in Turkish.

This approach is especially valuable for low-resource languages like Turkish, where high-quality domain-specific datasets are scarce. By bridging this gap with synthetic data, we aim to support the development of more accurate and inclusive financial AI tools for the Turkish-speaking community.

In [1]:
%%capture
!pip install langchain langchain_community  unstructured langchain_chroma langchain_chroma chromadb  langchain_ollama langchain_huggingface -q

In [4]:
import subprocess
import torch
from langchain_ollama import ChatOllama
from IPython.display import clear_output
import os

In [ ]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

!curl -fsSL https://ollama.com/install.sh | sh
subprocess.Popen("ollama serve", shell=True)

clear_output()

In [7]:
#subprocess.Popen("ollama pull gemma3n", shell=True)

llm = ChatOllama(
    model='gemma3n',
    temperature=0.2)

clear_output()

In [8]:
llm

ChatOllama(model='gemma3n', temperature=0.2, _client=<ollama._client.Client object at 0x000002586F126670>, _async_client=<ollama._client.AsyncClient object at 0x000002584CEF3E80>)

#### PDF Loader

In [11]:
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from pathlib import Path

#
pdf_url = "https://www.tcmb.gov.tr/wps/wcm/connect/2248323f-2e64-4850-9432-548087484aca/enf25_ii_tam.pdf?MOD=AJPERES&CACHEID=ROOTWORKSPACE-2248323f-2e64-4850-9432-548087484aca-puH-Dp5"

response = requests.get(pdf_url)
pdf_path = Path("downloaded_file.pdf")

with open(pdf_path, "wb") as f:
    f.write(response.content)

loader = PyPDFLoader(pdf_path)
docs = loader.load()

docs_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)
split_docs = docs_splitter.split_documents(docs)

"""
for doc in split_docs:
    print(doc)
"""

'\nfor doc in split_docs:\n    print(doc)\n'

#### Embeddings vector

In [ ]:
#create retriever folder
!mkdir retriever

In [15]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings 
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore  = Chroma.from_documents(split_docs , embeddings , persist_directory='/Tr-PDFtoSyntheticData/retriever')

retriever  = vectorstore.as_retriever()

TypeError: 'type' object is not subscriptable

In [8]:
#vectorstore
vectorstore = Chroma(
    persist_directory="/kaggle/working/retriever",
    embedding_function=embeddings
)

retriever = vectorstore.as_retriever()

/tmp/ipykernel_36/4253608616.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
import json 

In [17]:
q_prompt = """
Sen finansal okur yazarlık alanında insanlara bir şeyler öğretmek isteyen bir profesörsün. 
Elindeki kaynaklardan finans alanında genel olarak sorulması gereken mantıklı soruları çıkartırsın.
her bir sorunun arasına "~" koy.
Asla elindeki mentinden farklı bir konu bağlamaına çıkmaz ve gerekli soruları sorarsın. 
Bağlam:{context}
Soru: {input}
Cevap:
"""

q_prompt = ChatPromptTemplate.from_messages([
    ("system", q_prompt)
])

output_parser = StrOutputParser()

document_chain  = create_stuff_documents_chain(llm  =  llm , prompt = q_prompt , output_parser = output_parser )

rag_chain = create_retrieval_chain(retriever = retriever , combine_docs_chain = document_chain )

input_que = "bana 50 tane mantıklı soru üret"
response  = rag_chain.invoke({'input' : input_que})

sorular = response['answer'].split("~")[:-1]

[GIN] 2025/07/20 - 19:22:33 | 200 | 41.152140793s |       127.0.0.1 | POST     "/api/chat"


In [23]:
#sorular

In [22]:
system_message = """
Sen, finansal okuryazarlık seviyesi sıfır olan bireylere yardımcı olmak için tasarlanmış bir yapay zeka asistanısın.
Görevin, sadece verilen *bağlam* içindeki bilgilere dayanarak öğretici ve sade açıklamalar üretmektir.

Aşağıdaki JSON yapısını her zaman kullan:

{{
  "instruction": "Kullanıcının öğrenmek istediği finansal konu başlığı",
  "input": "Kullanıcının sorduğu basit soru",
  "output": "Konuyu hiç bilmeyen biri için kısa, açık, sade bir açıklama. Gerekirse günlük hayattan örnek vererek anlat."
{{

Kurallar:

1. Yalnızca verilen bağlamdaki bilgileri kullan. Bağlam dışında bilgi varsa, kesinlikle yanıt verme.

2. Asla uydurma bilgi üretme. Emin değilsen ya da bağlamda bilgi yoksa şu sabit yanıtı ver:
   "Bu konuda bilgim yok. Lütfen bir finansal danışmana başvurun."

3. Yanıtlar sade, kısa ve öğretici olsun. Teknik terimler gerekiyorsa açıklayıp günlük örnekle açıkla.
   Hedef kitlen, ilk kez finansla tanışan bireylerdir.

4. Format dışına çıkma. Her zaman belirtilen JSON yapısında cevap ver.

5. Bağlam dışı soruları pass geç ve o soruyu üretme. çıktı olarak sadece "Dökümantasyonla alakalı bir soru değil" de 

Bağlam: {context}
Soru:{input}
Cevap:
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message)
])

document_chain  = create_stuff_documents_chain(llm  =  llm , prompt = prompt , output_parser = output_parser )

rag_chain = create_retrieval_chain(retriever = retriever , combine_docs_chain = document_chain )

### Inferencing
#input_que = "Tarım sektörünün ilk çeyrekteki performansı nasıl değerlendirilebilir?"
#response  = rag_chain.invoke({'input' : input_que})

json_format = []

for i, soru in enumerate(sorular):
    response = rag_chain.invoke({'input': soru})
    print(response["answer"])
    json_format.append(response["answer"])
    print(f"{i}. Soru cevabı kayıt edildi")
    
with open("/kaggle/working/instruction_dataset.json", "w", encoding="utf-8") as f:
    json.dump(json_format, f, ensure_ascii=False, indent=4)


[GIN] 2025/07/20 - 19:24:30 | 200 |  7.455695876s |       127.0.0.1 | POST     "/api/chat"
{
 "instruction": "Kullanıcının öğrenmek isteği finansal konu başlığı",
 "input": "Enflasyonun temel nedenleri nelerdir?",
 "output": "Enflasyon, mal ve hizmetlerin fiyatlarındaki genel ve sürekli artıştır. Bunun birçok nedeni olabilir. Örneğin, çok fazla para basılması, üretim maliyetlerinin artması (hammadde, işçilik gibi) veya talebin arzı aşması enflasyona yol açabilir. Fiyatlar sürekli yükseldiğinde, aynı miktarda para ile daha az şey satın alabilirsiniz. Bu nedenle enflasyon, yaşam standartlarınızı etkileyebilir."
}

0. Soru cevabı kayıt edildi
[GIN] 2025/07/20 - 19:24:38 | 200 |  8.038045803s |       127.0.0.1 | POST     "/api/chat"
{
 "instruction": "Enflasyonun ekonomiye etkileri",
 "input": "Enflasyonuun ekonomiye etkileri nelerdir?",
 "outp": "Enflasyon, mal ve hizmetlerin fiyatlarındaki genel artış anlamına gelir. Bu durum ekonomiyi farklı şekillerde etkileyebilir. Örneğin, enflasyon 